In [1]:
from datasets import load_dataset
ashaar = load_dataset("arbml/ashaar")

Found cached dataset parquet (/home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# !pip install gdown
# !gdown https://drive.google.com/uc?id=18JcCCPwuvPVp4Tp3oeavjxzkJ056J_Fa
# !unzip final_model.zip

In [3]:
from models import create_transformer_model, char2idx
model = create_transformer_model()
model.load_weights("content/drive/MyDrive/Research/Barmajan/Models/ashaar_model_vocab_changed_diacritized_fixed4_transformer_bs256_50k_with_prose_augment_balanced_dedup/cp.ckpt")

In [4]:
with open('labels.txt', 'r') as f:
    label2name = f.readlines()
    label2name = [name.replace('\n', '') for name in label2name]

with open('labels_ar.txt', 'r') as f:
    label2name_ar = f.readlines()
    label2name_ar = [name.replace('\n', '') for name in label2name_ar]

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
def predict_meter(bayt):
    x = [[char2idx[char] for char in sent if char in char2idx] for sent in bayt]
    x = pad_sequences(x, padding='post', value=0, maxlen = 128)
    logits = model.predict(x)
    
    return Counter([label2name_ar[logit.argmax()]for logit in logits]).most_common(1)[0][0]
    

In [22]:
def process_verse(sample):
    chars = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويىئءأؤة ى'
    diacs = 'ْ~ًٌٍَُِّ'
    map_chars = {'ک':'ك', 'ﺑ':'ب', 'ٹ':'ث', 'ی':'ى'}
    out = []
    for verse in sample['poem verses']:
        proc_verse = ''
        for char in verse:
            if char in chars+diacs:
                proc_verse += char
            elif char in map_chars:
                proc_verse += map_chars[char]
        out.append(proc_verse)
    sample ['poem verses'] = out
    return sample

def filter_poems(sample):
    poem = sample['poem verses']
    if len(poem) < 2:
        return False
    if len(poem) % 2 != 0:
        return False
    for verse in poem:
        if len(verse) < 5:
            return False
    return True

In [7]:
ashaar = ashaar.map(process_verse)

Loading cached processed dataset at /home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7fefbbdb344e7589.arrow


In [8]:
ashaar = ashaar.filter(filter_poems)

Loading cached processed dataset at /home/alyafey22/.cache/huggingface/datasets/arbml___parquet/MagedSaeed--ashaar-719bb58a76ea0092/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-29e1c1a88312add9.arrow


In [9]:
ashaar

DatasetDict({
    train: Dataset({
        features: ['poem title', 'poem meter', 'poem verses', 'poem theme', 'poem url', 'poet name', 'poet description', 'poet url', 'poet era', 'poet location', 'poem description', 'poem language type'],
        num_rows: 219946
    })
})

In [10]:
ashaar['train'][2]['poem verses']

['لَو كُنتَ أَطمَع بِالمَنام تَوهما',
 'لَسالَت طَيفكَ أَن يَزور تَكَرما',
 'حاشا صُدودك أَن تَذم فَِنَّها',
 'تَحلو لَدَيَّ وَِن أُسيغَت عَلقَما',
 'فَاِهجُر فَهَجرك لي التِفات مَودة',
 'أَلقاهُ مِنكَ تَحنناً وَتَرَحُما',
 'عَذب فُوادي بِالَّذي تَختارَهُ',
 'لَو كُنتُ مَنسيّاً تَرَكتُ وَِنَّما',
 'لَو لَم تَكُن بِغُبار طَرفِكَ كَحلت',
 'عَين الغَزالة صَدّها وَجه الدُما',
 'عَيدي لِفَقدِكَ مَأَتم لَو صافَحت',
 'فيهِ المَسَرة خاطِري لَتَأَلَما',
 'هاتَ ِسقِني كَأس المَلامة عاذِلي',
 'وَأدر عَليَّ حَديثُهُ مُتَرَنِما',
 'فَِذا ذَكرت ليَ الحَبيب يَكادُ مِن',
 'طَربي يَقبل مَسمَعي مِنكَ الفَما',
 'ِني لَأَعشَقُ في هَواهُ عَواذِلي',
 'شَغَفاً بِهِ وَأَود فيهِ اللُوَّما',
 'سَرق الرَسول بَلَحظِهِ مِن وَجهِهِ',
 'حُسناً أَبى عَن ناظِري أَن يَكتما',
 'دَعني أُسامر هَجرَهُ في خُلوة',
 'فَكَفى لِمِثلي أَن يَراني مَحرَما',
 'بَدرٌ مِن الأَتراك لَما أَن بَدا',
 'تَرك البُدور تَرى لِعَينِكَ أَنجُما',
 'تَسقي لَواحِظُهُ العُقول مُدامة',
 'الصَحو مِنها لا يَزالُ محرّما',
 'لَو بِتُ أَشكو ظُلمُهُ لَشَ

In [23]:
def map_meters(sample):
    meter = sample['poem meter']
    verses = sample['poem verses'] 
    
    if meter:
        for label in label2name_ar:
            if label in meter:
                sample['poem meter'] = label
        if meter == 'بسيط':
            sample['poem meter'] = 'البسيط'
        if 'خبب' in meter:
            sample['poem meter'] = "المتدارك"
        if meter in ['نثرية', 'شعر التفعيلة', 'شعر الحر', 'بحر التفعيلة', 'التفعيله']:
            sample['poem meter'] = "النثر"
    if meter in ['عمودية', 'العمودية', None]:
        bayts = [verses[i] +' # '+ verses[i+1] for i in range(0, len(verses)-2, 2)]
        if len(bayts):
            sample['poem meter'] = predict_meter(bayts)

    return sample

filtered_meters = set()

def filter_meters(sample):
    meter = sample['poem meter']
    for m in label2name_ar:
        if m == meter:
            return True
    return False

In [24]:
ashaar = ashaar.map(map_meters)

INFO:tensorflow:Assets written to: ram://1f7e2122-0fdb-41d1-a3b1-732865f4a741/assets


INFO:tensorflow:Assets written to: ram://1f7e2122-0fdb-41d1-a3b1-732865f4a741/assets


Map:   0%|          | 0/219946 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

In [25]:
ashaar = ashaar.filter(filter_meters)

Filter:   0%|          | 0/219946 [00:00<?, ? examples/s]

In [26]:
ashaar

DatasetDict({
    train: Dataset({
        features: ['poem title', 'poem meter', 'poem verses', 'poem theme', 'poem url', 'poet name', 'poet description', 'poet url', 'poet era', 'poet location', 'poem description', 'poem language type'],
        num_rows: 212499
    })
})

In [31]:
meters = set(ashaar['train']['poem meter'])
meters

{'البسيط',
 'الخفيف',
 'الرجز',
 'الرمل',
 'السريع',
 'الطويل',
 'الكامل',
 'المتدارك',
 'المتقارب',
 'المجتث',
 'المديد',
 'المضارع',
 'المقتضب',
 'المنسرح',
 'النثر',
 'الهزج',
 'الوافر'}

In [73]:
meter_tokens = {meter:f'<|meter_{i}|>' for i, meter in enumerate(meters)}
print(meter_tokens)
with open("meter_tokens.json", "w") as f:
    json.dump(meter_tokens, f)

{'الخفيف': '<|meter_0|>', 'المضارع': '<|meter_1|>', 'المجتث': '<|meter_2|>', 'الرمل': '<|meter_3|>', 'البسيط': '<|meter_4|>', 'المتقارب': '<|meter_5|>', 'الوافر': '<|meter_6|>', 'المقتضب': '<|meter_7|>', 'المديد': '<|meter_8|>', 'النثر': '<|meter_9|>', 'الهزج': '<|meter_10|>', 'المتدارك': '<|meter_11|>', 'المنسرح': '<|meter_12|>', 'الطويل': '<|meter_13|>', 'الكامل': '<|meter_14|>', 'الرجز': '<|meter_15|>', 'السريع': '<|meter_16|>'}


In [39]:
themes = set(ashaar['train']['poem theme'])
themes

{None,
 'قصيدة اعتذار',
 'قصيدة الاناشيد',
 'قصيدة المعلقات',
 'قصيدة حزينه',
 'قصيدة دينية',
 'قصيدة ذم',
 'قصيدة رثاء',
 'قصيدة رومنسيه',
 'قصيدة سياسية',
 'قصيدة شوق',
 'قصيدة عامه',
 'قصيدة عتاب',
 'قصيدة غزل',
 'قصيدة فراق',
 'قصيدة قصيره',
 'قصيدة مدح',
 'قصيدة هجاء',
 'قصيدة وطنيه'}

In [72]:
import json
theme_tokens = {meter:f'<|theme_{i}|>' for i, meter in enumerate(themes)}
with open("theme_tokens.json", "w") as f:
    json.dump(theme_tokens, f)

In [35]:
ST_POEM_TOKEN = '<|psep|>'
ED_POEM_TOKEN = '</|psep|>'
VERSE_TOKEN = '<|vsep|>'
ST_BAYT_TOKEN= '<|bsep|>'
ED_BAYT_TOKEN= '</|bsep|>'
PAD_TOKEN = '<|pad|>'
EOT_TOKEN = '<|endoftext|>'

In [41]:
from bohour.qafiah import get_qafiyah
import collections

def get_qafiyah_majority(poem):
    all_qafiyahs =[]
    try:
        for bayt in poem.split(ED_BAYT_TOKEN)[:-1]:
            proc_bayt = bayt.replace(VERSE_TOKEN, '').replace(ST_BAYT_TOKEN, '').replace(ED_BAYT_TOKEN,'')
            all_qafiyahs.append(get_qafiyah([proc_bayt])[0][0])
        return collections.Counter(all_qafiyahs).most_common(1)[0][0]
    except:
        return ''
    
def join_verses(sample):

    verses = sample['poem verses']
    meter = sample['poem meter']
    theme = sample['poem theme']
  
    poem = ''.join([ f'{ST_BAYT_TOKEN} '+verses[i] +f' {VERSE_TOKEN} '+ verses[i+1]+ f' {ED_BAYT_TOKEN} ' for i in range(0, len(verses)-2, 2)])
    qafiyah = get_qafiyah_majority(poem)
    if meter is None:
        raise
    text = f"{meter_tokens[meter]} {qafiyah} {theme_tokens[theme]} {ST_POEM_TOKEN} {poem.strip()} {ED_POEM_TOKEN}"

    return {'text':text}

In [42]:
ashaar = ashaar.map(join_verses)

Map:   0%|          | 0/212499 [00:00<?, ? examples/s]

In [43]:
ashaar['train'][1]

{'poem title': 'من أي مولى ارتجي',
 'poem meter': 'الرمل',
 'poem verses': ['مِن أَي مَولى اِرتَجي',
  'وَلاي باب التَجي',
  'وَاللَهُ حَيٌّ رازِقٌ',
  'يُعطي الجَزيل لمُرتَجي',
  'رَب جَواد لَم يَزَل',
  'مِن كُل ضيقٍ مَخرَجي',
  'ِن رُحت أَرجوغَيرَهُ',
  'خابَ الرواح مَع المَجي',
  'يا عَيس مالي أَقصدي',
  'باب الكَريم وَعَرجي',
  'وَضَعي رِحالك وَاِرتَعي',
  'فَالأُم حَمل المُزعجِ',
  'وَتَوسَلي بِمُحمدٍ',
  'وَبِلهِ كَي تَنتجي',
  'الهاشمي المُصطَفى',
  'صج الهُدى المُتَبَلِجِ',
  'وَبِشَيبة الصَديق صا',
  'حب كل فَضل أَبهَجِ',
  'وَالسَيد الفاروق مِن',
  'بِسِوى الهُدى لَم يَلهجِ',
  'وَبصنوه عُثمان ذي الن',
  'نورَين أَقوم مَنهَجِ',
  'وَعَليٍّ الكرّار فا',
  'تح كُل باب مُرتجِ',
  'وَبَقية الصَحب الكِرا',
  'م أَولي الثَنا المُتَأرجِ',
  'هُم أَبحر الفَضل الَّذي',
  'نَ بِغَيرِهُم لَم تُفرجِ',
  'وَكَذا السَفينة ِن نَجَت',
  'فَجَميع مَن فيها نَجي'],
 'poem theme': 'قصيدة دينية',
 'poem url': 'https://www.aldiwan.net/poem16183.html',
 'poet name': 'الامير منجك باشا',
 'poet desc

In [44]:
batch_size = 1000

def batch_iterator():
    for i in range(0, len(ashaar['train']), batch_size):
        yield ashaar['train'][i : i + batch_size]["text"]

In [45]:
full_dataset = ""
for text in ashaar['train']["text"]:
    full_dataset += text
char_cnt =len(set(list(full_dataset)))

In [46]:
print(char_cnt)

68


In [47]:
print(set(list(full_dataset)))

{'أ', 'ي', 'ؤ', 's', 'ٍ', 'َ', 'ق', '5', 'ذ', 'r', 'ْ', 'ِ', 'ص', 'ع', 'ء', 'ث', '6', '~', 'm', '>', '|', 'ف', '7', '2', 'ه', ' ', 'و', 'ا', 'ب', 'ن', '1', 'ك', 'ش', 'غ', 'ّ', 'h', '0', 'ر', 'ة', 'ظ', 'س', 'e', '9', 'ئ', 'ً', 'م', 'p', 'ض', 'ز', '<', 'ت', 'v', 'خ', 'ى', '4', '8', 't', 'b', 'ط', 'ٌ', '3', '/', 'ل', 'ُ', 'د', 'ج', 'ح', '_'}


In [49]:
print(list(meter_tokens.values()))

['<|meter_0|>', '<|meter_1|>', '<|meter_2|>', '<|meter_3|>', '<|meter_4|>', '<|meter_5|>', '<|meter_6|>', '<|meter_7|>', '<|meter_8|>', '<|meter_9|>', '<|meter_10|>', '<|meter_11|>', '<|meter_12|>', '<|meter_13|>', '<|meter_14|>', '<|meter_15|>', '<|meter_16|>']


In [50]:
extra_tokens = [f'<|res_{i}|>' for i in range(10)]
print(extra_tokens)

['<|res_0|>', '<|res_1|>', '<|res_2|>', '<|res_3|>', '<|res_4|>', '<|res_5|>', '<|res_6|>', '<|res_7|>', '<|res_8|>', '<|res_9|>']


In [51]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
tokenizer = Tokenizer(models.BPE())
#This pre-tokenizer takes care of replacing all bytes of the given string with a corresponding representation, as well as splitting into words.
tokenizer.pre_tokenizer = pre_tokenizers.UnicodeScripts() #add space whether to add space at the first word
special_tokens = [EOT_TOKEN, ST_POEM_TOKEN, ED_POEM_TOKEN, VERSE_TOKEN, ST_BAYT_TOKEN, ED_BAYT_TOKEN, PAD_TOKEN]+list(meter_tokens.values())+list(theme_tokens.values())+extra_tokens
trainer = trainers.BpeTrainer(vocab_size=char_cnt+len(special_tokens), special_tokens=special_tokens)
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)
tokenizer.pad_token = PAD_TOKEN
print(tokenizer.get_vocab_size())




121


In [52]:
from transformers import GPT2TokenizerFast, AutoTokenizer
gpt_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [53]:
print([(gpt_tokenizer.tokenize(sp)[0], sp) for sp in special_tokens])

[('<|endoftext|>', '<|endoftext|>'), ('<|psep|>', '<|psep|>'), ('</|psep|>', '</|psep|>'), ('<|vsep|>', '<|vsep|>'), ('<|bsep|>', '<|bsep|>'), ('</|bsep|>', '</|bsep|>'), ('<|pad|>', '<|pad|>'), ('<|meter_0|>', '<|meter_0|>'), ('<|meter_1|>', '<|meter_1|>'), ('<|meter_2|>', '<|meter_2|>'), ('<|meter_3|>', '<|meter_3|>'), ('<|meter_4|>', '<|meter_4|>'), ('<|meter_5|>', '<|meter_5|>'), ('<|meter_6|>', '<|meter_6|>'), ('<|meter_7|>', '<|meter_7|>'), ('<|meter_8|>', '<|meter_8|>'), ('<|meter_9|>', '<|meter_9|>'), ('<|meter_10|>', '<|meter_10|>'), ('<|meter_11|>', '<|meter_11|>'), ('<|meter_12|>', '<|meter_12|>'), ('<|meter_13|>', '<|meter_13|>'), ('<|meter_14|>', '<|meter_14|>'), ('<|meter_15|>', '<|meter_15|>'), ('<|meter_16|>', '<|meter_16|>'), ('<|theme_0|>', '<|theme_0|>'), ('<|theme_1|>', '<|theme_1|>'), ('<|theme_2|>', '<|theme_2|>'), ('<|theme_3|>', '<|theme_3|>'), ('<|theme_4|>', '<|theme_4|>'), ('<|theme_5|>', '<|theme_5|>'), ('<|theme_6|>', '<|theme_6|>'), ('<|theme_7|>', '<|them

In [54]:
len(gpt_tokenizer.get_vocab())

121

In [55]:
ashaar.push_to_hub('ashaar_datasetv2')

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/107 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
gpt_tokenizer.push_to_hub('ashaar_tokenizerv2')

CommitInfo(commit_url='https://huggingface.co/Zaid/ashaar_tokenizerv2/commit/5aa42fc45d97ddee91c44dcb21c07e6d01f46f6b', commit_message='Upload tokenizer', commit_description='', oid='5aa42fc45d97ddee91c44dcb21c07e6d01f46f6b', pr_url=None, pr_revision=None, pr_num=None)

In [57]:
from transformers import GPT2TokenizerFast, AutoTokenizer

gpt_tokenizer = GPT2TokenizerFast.from_pretrained('Zaid/ashaar_tokenizerv2')

In [58]:
print(gpt_tokenizer.get_vocab())

{'<|theme_0|>': 24, '<|theme_4|>': 28, ' ': 53, '<|res_3|>': 46, 'ش': 96, 'ة': 85, '<|meter_2|>': 9, '<|theme_11|>': 35, '5': 60, 'ٍ': 115, 'س': 95, 'ٌ': 114, '<|meter_8|>': 15, 'ِ': 118, 'ّ': 119, '<|theme_7|>': 31, 't': 75, '<|res_5|>': 48, '<|endoftext|>': 0, '<|theme_1|>': 25, 'ى': 111, '<|theme_2|>': 26, 'ق': 104, '<|meter_7|>': 14, '<|theme_14|>': 38, '<|res_2|>': 45, 'ت': 86, '|': 77, '<|theme_15|>': 39, 'ل': 106, '<|meter_0|>': 7, 'أ': 80, '<|meter_9|>': 16, '<|theme_17|>': 41, '<|meter_5|>': 12, '<|theme_10|>': 34, '<|theme_6|>': 30, '<|theme_12|>': 36, 'ؤ': 81, 'h': 70, 'ز': 94, 'ي': 112, '<|theme_16|>': 40, 'خ': 90, 'ن': 108, '</|bsep|>': 5, 'ط': 99, 'ف': 103, '<|theme_9|>': 33, '7': 62, 'ء': 79, 'm': 71, '<|vsep|>': 3, '<|pad|>': 6, 'ص': 97, 'ه': 109, '<|meter_4|>': 11, '<|bsep|>': 4, 'ئ': 82, '<|meter_14|>': 21, '/': 54, '3': 58, 'و': 110, '8': 63, '<|meter_1|>': 8, '<|meter_13|>': 20, 'r': 73, 'ا': 83, '<|res_4|>': 47, '<|meter_15|>': 22, '_': 67, 'ظ': 100, '<|meter_12|>'

In [64]:
gpt_tokenizer.encode('<|theme_0|>')

[24]